In [1]:
import numpy as np
import pandas as pd
import folium
import shapefile
from collections import OrderedDict
import numpy as np 
import pandas as pd
import json
import requests 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
myshp = open("./TrainStation_Oct2017/MRTLRTStnPtt.shp", 'rb')
mydbf = open("./TrainStation_Oct2017/MRTLRTStnPtt.dbf", 'rb')
sf = shapefile.Reader(shp=myshp, dbf=mydbf)
records = sf.shapeRecords()

# Print first 5 records.
for x in records[:5]:
    msg = '{0} ({1})  @  {2}'
    print(msg.format(x.record[1], x.record[2], x.shape.points[0]))

EUNOS MRT STATION (EW7)  @  [35782.955299999565, 33560.077600000426]
CHINESE GARDEN MRT STATION (EW25)  @  [16790.746600000188, 36056.30189999938]
KHATIB MRT STATION (NS14)  @  [27962.310800000094, 44352.56799999997]
KRANJI MRT STATION (NS7)  @  [20081.697399999946, 45214.54790000059]
REDHILL MRT STATION (EW18)  @  [26163.47800000012, 30218.819599999115]


In [3]:
# Retrieve the station name, station code, and the coordinates
tuple_list = [(x.record[1], x.record[2], x.shape.points[0]) for x in records]

# Unzip list of tuples into individual lists
stn_names, stn_codes, coordinates = zip(*tuple_list)

d = OrderedDict([('Station', stn_names), 
                 ('Code', stn_codes), 
                 ('SVY21_Coordinates', coordinates)])

df = pd.DataFrame(d)
df.head()

,Station,Code,SVY21_Coordinates
0,EUNOS MRT STATION,EW7,"[35782.955299999565, 33560.077600000426]"
1,CHINESE GARDEN MRT STATION,EW25,"[16790.746600000188, 36056.30189999938]"
2,KHATIB MRT STATION,NS14,"[27962.310800000094, 44352.56799999997]"
3,KRANJI MRT STATION,NS7,"[20081.697399999946, 45214.54790000059]"
4,REDHILL MRT STATION,EW18,"[26163.47800000012, 30218.819599999115]"


In [4]:
# Importing a module to convert SVY21 to standard longitude/latitude.
# https://gist.githubusercontent.com/jovianlin/30b0fd93e2b835cb1e8d93bfa0e1e62f/raw/89548360fb54ab1e0441b0935fc52fbbac6d263e/SVY21.py
from svy21 import SVY21

In [5]:
df['Coordinates'] = df.SVY21_Coordinates.apply(lambda x: SVY21().computeLatLon(x[1], x[0]))
df = df.drop('SVY21_Coordinates', axis=1)
df.head()

,Station,Code,Coordinates
0,EUNOS MRT STATION,EW7,"(1.3197782848869883, 103.90325213335886)"
1,CHINESE GARDEN MRT STATION,EW25,"(1.3423521542087522, 103.73259640485367)"
2,KHATIB MRT STATION,NS14,"(1.4173827034869275, 103.83297957440541)"
3,KRANJI MRT STATION,NS7,"(1.4251770321043908, 103.76216507762184)"
4,REDHILL MRT STATION,EW18,"(1.2895620597351773, 103.81681633682066)"


In [6]:
# Initial map.
map_sg = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

for _, row in df.iterrows():
    stn_name = row.Station
    stn_code = row.Code
    lat, lng = row.Coordinates
    label = '{} ({})'.format(stn_name, stn_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False).add_to(map_sg)  

map_sg

In [7]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500

def getNearbyVenues(df, radius=500):
    
    venues_list=[]
    
    for _, row in df.iterrows():
        name = row.Station
        lat, lng = row.Coordinates
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

sg_venues = getNearbyVenues(df)

EUNOS MRT STATION
CHINESE GARDEN MRT STATION
KHATIB MRT STATION
KRANJI MRT STATION
REDHILL MRT STATION
YEW TEE MRT STATION
PIONEER MRT STATION
COMMONWEALTH MRT STATION
PAYA LEBAR MRT STATION
SIMEI MRT STATION
TAMPINES MRT STATION
ADMIRALTY MRT STATION
YISHUN MRT STATION
PASIR RIS MRT STATION
CLEMENTI MRT STATION
ALJUNIED MRT STATION
ANG MO KIO MRT STATION
EXPO MRT STATION
CHOA CHU KANG MRT STATION
JURONG EAST MRT STATION
QUEENSTOWN MRT STATION
TUAS LINK MRT STATION
GUL CIRCLE MRT STATION
WOODLANDS MRT STATION
TUAS CRESCENT MRT STATION
KALLANG MRT STATION
TUAS WEST ROAD MRT STATION
BUONA VISTA MRT STATION
DOVER MRT STATION
SEMBAWANG MRT STATION
KEMBANGAN MRT STATION
BUKIT BATOK MRT STATION
ANG MO KIO MRT STATION
BOON LAY MRT STATION
BUKIT GOMBAK MRT STATION
MARSILING MRT STATION
BEDOK MRT STATION
JOO KOON MRT STATION
LAKESIDE MRT STATION
TANAH MERAH MRT STATION
YIO CHU KANG MRT STATION
OASIS LRT STATION
SAM KEE LRT STATION
FARMWAY LRT STATION
KUPANG LRT STATION
RENJONG LRT STATION
BUKIT

In [8]:
sg_venues.head()

,Station,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,EUNOS MRT STATION,1.319778,103.903252,福春叻沙卤面 Laksa & Lor Mee,1.320254,103.904132,Noodle House
1,EUNOS MRT STATION,1.319778,103.903252,Hai Kee Soy Sauce Chicken Rice,1.321457,103.902524,Coffee Shop
2,EUNOS MRT STATION,1.319778,103.903252,GYMM BOXX,1.317912,103.901094,Gym
3,EUNOS MRT STATION,1.319778,103.903252,香港街珍達記,1.316703,103.901415,Chinese Restaurant
4,EUNOS MRT STATION,1.319778,103.903252,LiHO,1.319729,103.902512,Bubble Tea Shop


In [9]:
# One hot encoding.
sg_onehot = pd.get_dummies(sg_venues[['Venue Category']], prefix="", prefix_sep="")

# Add station column back to dataframe.
sg_onehot['Station'] = sg_venues['Station'] 

# Move station column to the first column.
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

sg_onehot.head()

,Station,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Warehouse Store,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,EUNOS MRT STATION,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,EUNOS MRT STATION,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,EUNOS MRT STATION,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,EUNOS MRT STATION,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,EUNOS MRT STATION,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
sg_grouped = sg_onehot.groupby('Station').mean().reset_index()
sg_grouped.head()

,Station,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Warehouse Store,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,ADMIRALTY MRT STATION,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALJUNIED MRT STATION,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ANG MO KIO MRT STATION,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BAKAU LRT STATION,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BANGKIT LRT STATION,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
num_top_venues = 5

for stn in sg_grouped['Station']:
    print("----"+stn+"----")
    temp = sg_grouped[sg_grouped['Station'] == stn].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ADMIRALTY MRT STATION----
                  venue  freq
0           Supermarket  0.22
1          Burger Joint  0.11
2  Fast Food Restaurant  0.11
3            Food Court  0.11
4           Bus Station  0.11


----ALJUNIED MRT STATION----
                           venue  freq
0             Chinese Restaurant  0.12
1               Asian Restaurant  0.08
2                     Food Court  0.08
3  Vegetarian / Vegan Restaurant  0.06
4                           Café  0.06


----ANG MO KIO MRT STATION----
                 venue  freq
0          Coffee Shop  0.10
1           Food Court  0.07
2          Supermarket  0.05
3         Dessert Shop  0.05
4  Japanese Restaurant  0.05


----BAKAU LRT STATION----
                  venue  freq
0  Fast Food Restaurant   0.1
1    Light Rail Station   0.1
2           Bus Station   0.1
3          Betting Shop   0.1
4                 Trail   0.1


----BANGKIT LRT STATION----
                venue  freq
0          Food Court  0.17
1         Supermarket  0

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues.
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe.
station_venues_sorted = pd.DataFrame(columns=columns)
station_venues_sorted['Station'] = sg_grouped['Station']

for ind in np.arange(sg_grouped.shape[0]):
    station_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sg_grouped.iloc[ind, :], num_top_venues)

len(station_venues_sorted)

163

In [14]:
# Set number of clusters.
kclusters = 5

sg_grouped_clustering = sg_grouped.drop('Station', 1)

# Run k-means clustering.
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_grouped_clustering)

# Check cluster labels generated for each row in the dataframe.
kmeans.labels_[0:10] 

array([1, 2, 1, 0, 1, 0, 0, 0, 0, 0])

In [15]:
# Add clustering labels.
station_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Merge station_venues_sorted with df.
sg_merged = station_venues_sorted.merge(df, on="Station")

sg_merged.head()

,Cluster Labels,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Code,Coordinates
0,1,ADMIRALTY MRT STATION,Supermarket,Burger Joint,Bus Stop,Bus Station,Tea Room,Fast Food Restaurant,Food Court,Coffee Shop,Food Stand,Food Truck,NS10,"(1.44058433441903, 103.800987667492)"
1,2,ALJUNIED MRT STATION,Chinese Restaurant,Food Court,Asian Restaurant,Café,Noodle House,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Seafood Restaurant,Coffee Shop,Kitchen Supply Store,EW9,"(1.3164319449909239, 103.88290571130439)"
2,1,ANG MO KIO MRT STATION,Coffee Shop,Food Court,Bubble Tea Shop,Fast Food Restaurant,Snack Place,Dessert Shop,Sandwich Place,Supermarket,Japanese Restaurant,Convenience Store,NS16,"(1.369346637121707, 103.84993266055184)"
3,1,ANG MO KIO MRT STATION,Coffee Shop,Food Court,Bubble Tea Shop,Fast Food Restaurant,Snack Place,Dessert Shop,Sandwich Place,Supermarket,Japanese Restaurant,Convenience Store,NS16,"(1.369932508571624, 103.84955779810016)"
4,0,BAKAU LRT STATION,Supermarket,Japanese Restaurant,Betting Shop,Trail,Sandwich Place,Light Rail Station,Bus Station,Plaza,Fast Food Restaurant,Dessert Shop,SE3,"(1.388092037108859, 103.90543877085315)"


In [16]:
# Create final map.
map_clusters = folium.Map(location=[1.3521, 103.8198], zoom_start=11)

# Set color scheme for the clusters.
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map.
markers_colors = []
for _, row in sg_merged.iterrows():
    poi = row.Station
    cluster = row["Cluster Labels"]
    lat, lon = row.Coordinates
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.vector_layers.Circle(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [17]:
sg_merged.loc[sg_merged['Cluster Labels'] == 0, sg_merged.columns[[1] + list(range(2, sg_merged.shape[1]-2))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,BAKAU LRT STATION,Supermarket,Japanese Restaurant,Betting Shop,Trail,Sandwich Place,Light Rail Station,Bus Station,Plaza,Fast Food Restaurant,Dessert Shop
6,BARTLEY MRT STATION,Bus Station,Noodle House,Indian Restaurant,Metro Station,Café,Bus Stop,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop
7,BAYFRONT MRT STATION,Hotel,Boutique,Theater,Japanese Restaurant,Noodle House,Tea Room,Bar,Bridge,Waterfront,Design Studio
8,BEAUTY WORLD MRT STATION,Chinese Restaurant,Korean Restaurant,Café,Food Court,Coffee Shop,Pizza Place,Hainan Restaurant,Shopping Mall,Noodle House,Massage Studio
9,BEDOK MRT STATION,Coffee Shop,Japanese Restaurant,Sandwich Place,Chinese Restaurant,Food Court,Supermarket,Restaurant,Fast Food Restaurant,Sushi Restaurant,Asian Restaurant
10,BEDOK NORTH MRT STATION,Indian Restaurant,Food Court,Chinese Restaurant,Bus Station,Bus Line,Food & Drink Shop,Fast Food Restaurant,Convenience Store,Sandwich Place,Skate Park
12,BENCOOLEN MRT STATION,Café,Hotel,Japanese Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Asian Restaurant,Sandwich Place,Park,Food Court,Coffee Shop
13,BENDEMEER MRT STATION,Hostel,BBQ Joint,Coffee Shop,Restaurant,Supermarket,Bar,Soup Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant
17,BOON LAY MRT STATION,Japanese Restaurant,Fast Food Restaurant,Asian Restaurant,Dessert Shop,Chinese Restaurant,Coffee Shop,Convenience Store,Electronics Store,Café,Massage Studio
18,BOTANIC GARDENS MRT STATION,Café,Bakery,Noodle House,Shopping Mall,Gas Station,Thai Restaurant,Malay Restaurant,Satay Restaurant,Fast Food Restaurant,Metro Station


In [18]:
sg_merged.loc[sg_merged['Cluster Labels'] == 1, sg_merged.columns[[1] + list(range(2, sg_merged.shape[1]-2))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADMIRALTY MRT STATION,Supermarket,Burger Joint,Bus Stop,Bus Station,Tea Room,Fast Food Restaurant,Food Court,Coffee Shop,Food Stand,Food Truck
2,ANG MO KIO MRT STATION,Coffee Shop,Food Court,Bubble Tea Shop,Fast Food Restaurant,Snack Place,Dessert Shop,Sandwich Place,Supermarket,Japanese Restaurant,Convenience Store
3,ANG MO KIO MRT STATION,Coffee Shop,Food Court,Bubble Tea Shop,Fast Food Restaurant,Snack Place,Dessert Shop,Sandwich Place,Supermarket,Japanese Restaurant,Convenience Store
5,BANGKIT LRT STATION,Supermarket,Food Court,Pharmacy,Bike Trail,Miscellaneous Shop,Park,Video Store,Coffee Shop,Noodle House,Market
11,BEDOK RESERVOIR MRT STATION,Reservoir,Asian Restaurant,Dessert Shop,Park,Bus Station,Food Court,Noodle House,Supermarket,Hospital,Flea Market
14,BISHAN MRT STATION,Coffee Shop,Chinese Restaurant,Bubble Tea Shop,Food Court,Cosmetics Shop,Supermarket,Café,Japanese Restaurant,Ice Cream Shop,Electronics Store
15,BISHAN MRT STATION,Coffee Shop,Chinese Restaurant,Bubble Tea Shop,Food Court,Cosmetics Shop,Supermarket,Café,Japanese Restaurant,Ice Cream Shop,Electronics Store
25,BUKIT BATOK MRT STATION,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Food Court,Grocery Store,Frozen Yogurt Shop,Basketball Court,Skate Park,Sandwich Place,Mobile Phone Shop
27,BUKIT GOMBAK MRT STATION,Food Court,Stadium,Coffee Shop,Chinese Restaurant,Flea Market,Malay Restaurant,Sandwich Place,Shoe Store,Bubble Tea Shop,Fast Food Restaurant
28,BUKIT PANJANG LRT STATION,Coffee Shop,Bus Station,Fast Food Restaurant,Asian Restaurant,Sushi Restaurant,Korean Restaurant,Bubble Tea Shop,American Restaurant,Café,Shopping Mall


In [19]:
sg_merged.loc[sg_merged['Cluster Labels'] == 2, sg_merged.columns[[1] + list(range(2, sg_merged.shape[1]-2))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,ALJUNIED MRT STATION,Chinese Restaurant,Food Court,Asian Restaurant,Café,Noodle House,Vegetarian / Vegan Restaurant,Dim Sum Restaurant,Seafood Restaurant,Coffee Shop,Kitchen Supply Store
16,BOON KENG MRT STATION,Chinese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Seafood Restaurant,Asian Restaurant,Fast Food Restaurant,Shopping Mall,Electronics Store
20,BRADDELL MRT STATION,Chinese Restaurant,Noodle House,Food Court,Café,Fast Food Restaurant,Seafood Restaurant,Thai Restaurant,Hakka Restaurant,Bakery,Asian Restaurant
38,CHINESE GARDEN MRT STATION,Chinese Restaurant,Coffee Shop,Indian Restaurant,Food Court,Café,Train Station,Playground,Asian Restaurant,Food Stand,Food & Drink Shop
44,COMMONWEALTH MRT STATION,Chinese Restaurant,Noodle House,Bus Station,Asian Restaurant,Vegetarian / Vegan Restaurant,Ice Cream Shop,Train Station,Garden,Food Court,Supermarket
48,DAKOTA MRT STATION,Noodle House,Chinese Restaurant,Seafood Restaurant,BBQ Joint,Dessert Shop,Asian Restaurant,Bar,Food Court,Coffee Shop,Steakhouse
56,EUNOS MRT STATION,Chinese Restaurant,Noodle House,Asian Restaurant,Breakfast Spot,Food Court,Gym,Seafood Restaurant,Train Station,Coffee Shop,Vegetarian / Vegan Restaurant
62,FARRER ROAD MRT STATION,Chinese Restaurant,Indonesian Restaurant,Gym / Fitness Center,Noodle House,Convenience Store,Betting Shop,Pool,Food Court,Bakery,Playground
81,KEMBANGAN MRT STATION,Indian Restaurant,Chinese Restaurant,Bus Stop,Coffee Shop,Supermarket,Noodle House,Food Court,Snack Place,Bistro,Seafood Restaurant
85,KOVAN MRT STATION,Chinese Restaurant,Food Court,Noodle House,Café,Asian Restaurant,Coffee Shop,Dessert Shop,Bakery,Supermarket,Bistro


In [20]:
sg_merged.loc[sg_merged['Cluster Labels'] == 3, sg_merged.columns[[1] + list(range(2, sg_merged.shape[1]-2))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,CASHEW MRT STATION,Food Court,Convenience Store,Seafood Restaurant,Basketball Court,Bus Station,Yoga Studio,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant
73,JELAPANG LRT STATION,Food Court,Supermarket,Gun Range,Convenience Store,Park,Smoke Shop,Basketball Court,Fishing Spot,Fast Food Restaurant,Field
119,PENDING LRT STATION,Food Court,Market,Miscellaneous Shop,Park,Noodle House,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market
149,SOO TECK LRT STATION,Food Court,Tennis Court,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
177,WOODLANDS SOUTH MRT STATION,Food Court,Video Store,Dessert Shop,Park,Trail,Bus Station,Basketball Court,Food Stand,Food Truck,Food & Drink Shop


In [21]:
sg_merged.loc[sg_merged['Cluster Labels'] == 4, sg_merged.columns[[1] + list(range(2, sg_merged.shape[1]-2))]]

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
128,PUNGGOL POINT LRT STATION,Light Rail Station,Campground,Yoga Studio,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market
